In [1]:
import getpass
import os
from langchain_core.tools import tool
from langchain_deepseek import ChatDeepSeek

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

_set_if_undefined("DEEPSEEK_API_KEY")

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=10, 
)

In [2]:
# Schema for structured output
from pydantic import BaseModel, Field

class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Query that is optimized web search.")
    justification: str = Field(
        None, description="Why this query is relevant to the user's request."
    )


# Augment the LLM with schema for structured output
structured_llm = llm.with_structured_output(SearchQuery)

# Invoke the augmented LLM
output = structured_llm.invoke("How does Calcium CT score relate to high cholesterol?")

# Define a tool
def multiply(a: int, b: int) -> int:
    return a * b

# Augment the LLM with tools
llm_with_tools = llm.bind_tools([multiply])

# Invoke the LLM with input that triggers the tool call
msg = llm_with_tools.invoke("What is 2 times 3?")

# Get the tool call
msg.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'call_0_b39ac643-848e-4663-a8fe-3cc8747048fc',
  'type': 'tool_call'}]

In [3]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class State(TypedDict):
    topic: str
    joke: str
    improved_joke: str
    final_joke: str


# Nodes
def generate_joke(state: State):
    """First LLM call to generate initial joke"""

    msg = llm.invoke(f"Write a short joke about {state['topic']}")
    return {"joke": msg.content}


def check_punchline(state: State):
    """Gate function to check if the joke has a punchline"""

    # Simple check - does the joke contain "?" or "!"
    if "?" in state["joke"] or "!" in state["joke"]:
        return "Fail"
    return "Pass"


def improve_joke(state: State):
    """Second LLM call to improve the joke"""

    msg = llm.invoke(f"Make this joke funnier by adding wordplay: {state['joke']}")
    return {"improved_joke": msg.content}


def polish_joke(state: State):
    """Third LLM call for final polish"""

    msg = llm.invoke(f"Add a surprising twist to this joke: {state['improved_joke']}")
    return {"final_joke": msg.content}


# Build workflow
workflow = StateGraph(State)

# Add nodes
workflow.add_node("generate_joke", generate_joke)
workflow.add_node("improve_joke", improve_joke)
workflow.add_node("polish_joke", polish_joke)

# Add edges to connect nodes
workflow.add_edge(START, "generate_joke")
workflow.add_conditional_edges(
    "generate_joke", check_punchline, {"Fail": "improve_joke", "Pass": END}
)
workflow.add_edge("improve_joke", "polish_joke")
workflow.add_edge("polish_joke", END)

# Compile
chain = workflow.compile()

# Invoke
state = chain.invoke({"topic": "cats"})
print("Initial joke:")
print(state["joke"])
print("\n--- --- ---\n")
if "improved_joke" in state:
    print("Improved joke:")
    print(state["improved_joke"])
    print("\n--- --- ---\n")

    print("Final joke:")
    print(state["final_joke"])
else:
    print("Joke failed quality gate - no punchline detected!")

Initial joke:
Sure! Here's a quick one:  

**Why don’t cats play poker in the wild?**  
*Too many cheetahs!*  

(Or if you prefer a classic: **"I told my cat I was reading a book about anti-gravity. He couldn’t put it down."**)  

Hope that gives you a chuckle! 😸

--- --- ---

Improved joke:
Here’s a funnier version with extra wordplay:  

**Why don’t cats play poker in the wild?**  
*Too many cheetahs—they always *spotted* the bluff!*  

And for the classic:  
**"I told my cat I was reading a book about anti-gravity. He couldn’t put it down—guess he’s *paws-itively* hooked!"**  

Now it’s a *purr-fect* combo of puns! 😸🎲

--- --- ---

Final joke:
Here’s a *wildly* unexpected twist to your already hilarious jokes—now with an extra layer of absurdity and wordplay:  

---  

**Why don’t cats play poker in the wild?**  
*Too many cheetahs—they always *spotted* the bluff!*  
**But the real reason?** *The ante-lopes kept raising the stakes.* 🦌🎰  

---  

**"I told my cat I was reading a book

In [4]:
# Graph state
class State(TypedDict):
    topic: str
    joke: str
    story: str
    poem: str
    combined_output: str


# Nodes
def call_llm_1(state: State):
    """First LLM call to generate initial joke"""

    msg = llm.invoke(f"Write a joke about {state['topic']}")
    return {"joke": msg.content}


def call_llm_2(state: State):
    """Second LLM call to generate story"""

    msg = llm.invoke(f"Write a story about {state['topic']}")
    return {"story": msg.content}


def call_llm_3(state: State):
    """Third LLM call to generate poem"""

    msg = llm.invoke(f"Write a poem about {state['topic']}")
    return {"poem": msg.content}


def aggregator(state: State):
    """Combine the joke and story into a single output"""

    combined = f"Here's a story, joke, and poem about {state['topic']}!\n\n"
    combined += f"STORY:\n{state['story']}\n\n"
    combined += f"JOKE:\n{state['joke']}\n\n"
    combined += f"POEM:\n{state['poem']}"
    return {"combined_output": combined}


# Build workflow
parallel_builder = StateGraph(State)

# Add nodes
parallel_builder.add_node("call_llm_1", call_llm_1)
parallel_builder.add_node("call_llm_2", call_llm_2)
parallel_builder.add_node("call_llm_3", call_llm_3)
parallel_builder.add_node("aggregator", aggregator)

# Add edges to connect nodes
parallel_builder.add_edge(START, "call_llm_1")
parallel_builder.add_edge(START, "call_llm_2")
parallel_builder.add_edge(START, "call_llm_3")
parallel_builder.add_edge("call_llm_1", "aggregator")
parallel_builder.add_edge("call_llm_2", "aggregator")
parallel_builder.add_edge("call_llm_3", "aggregator")
parallel_builder.add_edge("aggregator", END)
parallel_workflow = parallel_builder.compile()


# Invoke
state = parallel_workflow.invoke({"topic": "cats"})
print(state["combined_output"])

Here's a story, joke, and poem about cats!

STORY:
**The Secret Kingdom of Whiskerfell**  

In a quiet little town, nestled between rolling hills and dense forests, there was a place known only to cats—**Whiskerfell**, a hidden kingdom where felines ruled with grace and cunning.  

By day, the cats of the town were ordinary pets—napping in sunbeams, chasing yarn, and occasionally knocking things off tables. But when the moon rose high, they slipped through hidden doorways, vanishing into the shadows.  

One night, a curious kitten named **Pip** followed his older sister, Luna, as she darted behind an old oak tree. To his amazement, the bark shimmered, and he tumbled into a world of towering catnip castles, glowing firefly lanterns, and cats of all shapes and sizes gathered in a grand council.  

"Who dares enter Whiskerfell uninvited?" rumbled **Lord Tiberius**, a massive Maine Coon with a golden mane.  

Luna sighed. "This is Pip, my little brother. He followed me."  

The council mur

In [5]:
from typing_extensions import Literal
from langchain_core.messages import HumanMessage, SystemMessage


# Schema for structured output to use as routing logic
class Route(BaseModel):
    step: Literal["poem", "story", "joke"] = Field(
        None, description="The next step in the routing process"
    )


# Augment the LLM with schema for structured output
router = llm.with_structured_output(Route)


# State
class State(TypedDict):
    input: str
    decision: str
    output: str


# Nodes
def llm_call_1(state: State):
    """Write a story"""

    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_2(state: State):
    """Write a joke"""

    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_3(state: State):
    """Write a poem"""

    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_router(state: State):
    """Route the input to the appropriate node"""

    # Run the augmented LLM with structured output to serve as routing logic
    decision = router.invoke(
        [
            SystemMessage(
                content="Route the input to story, joke, or poem based on the user's request."
            ),
            HumanMessage(content=state["input"]),
        ]
    )

    return {"decision": decision.step}


# Conditional edge function to route to the appropriate node
def route_decision(state: State):
    # Return the node name you want to visit next
    if state["decision"] == "story":
        return "llm_call_1"
    elif state["decision"] == "joke":
        return "llm_call_2"
    elif state["decision"] == "poem":
        return "llm_call_3"


# Build workflow
router_builder = StateGraph(State)

# Add nodes
router_builder.add_node("llm_call_1", llm_call_1)
router_builder.add_node("llm_call_2", llm_call_2)
router_builder.add_node("llm_call_3", llm_call_3)
router_builder.add_node("llm_call_router", llm_call_router)

# Add edges to connect nodes
router_builder.add_edge(START, "llm_call_router")
router_builder.add_conditional_edges(
    "llm_call_router",
    route_decision,
    {  # Name returned by route_decision : Name of next node to visit
        "llm_call_1": "llm_call_1",
        "llm_call_2": "llm_call_2",
        "llm_call_3": "llm_call_3",
    },
)
router_builder.add_edge("llm_call_1", END)
router_builder.add_edge("llm_call_2", END)
router_builder.add_edge("llm_call_3", END)

# Compile workflow
router_workflow = router_builder.compile()

# Invoke
state = router_workflow.invoke({"input": "Write me a joke about cats"})
print(state["output"])

Sure! Here's a purr-fect cat joke for you:  

**Why don’t cats play poker in the jungle?**  
*Because there are too many cheetahs!* 🐆😹  

Let me know if you want more—I’ve got a *litter* of them! 😸


In [6]:
from typing import Annotated, List
import operator


# Schema for structured output to use in planning
class Section(BaseModel):
    name: str = Field(
        description="Name for this section of the report.",
    )
    description: str = Field(
        description="Brief overview of the main topics and concepts to be covered in this section.",
    )


class Sections(BaseModel):
    sections: List[Section] = Field(
        description="Sections of the report.",
    )


# Augment the LLM with schema for structured output
planner = llm.with_structured_output(Sections)

In [8]:
from langgraph.constants import Send


# Graph state
class State(TypedDict):
    topic: str  # Report topic
    sections: list[Section]  # List of report sections
    completed_sections: Annotated[
        list, operator.add
    ]  # All workers write to this key in parallel
    final_report: str  # Final report


# Worker state
class WorkerState(TypedDict):
    section: Section
    completed_sections: Annotated[list, operator.add]


# Nodes
def orchestrator(state: State):
    """Orchestrator that generates a plan for the report"""

    # Generate queries
    report_sections = planner.invoke(
        [
            SystemMessage(content="Generate a plan for the report."),
            HumanMessage(content=f"Here is the report topic: {state['topic']}"),
        ]
    )

    return {"sections": report_sections.sections}


def llm_call(state: WorkerState):
    """Worker writes a section of the report"""

    # Generate section
    section = llm.invoke(
        [
            SystemMessage(
                content="Write a report section following the provided name and description. Include no preamble for each section. Use markdown formatting."
            ),
            HumanMessage(
                content=f"Here is the section name: {state['section'].name} and description: {state['section'].description}"
            ),
        ]
    )

    # Write the updated section to completed sections
    return {"completed_sections": [section.content]}


def synthesizer(state: State):
    """Synthesize full report from sections"""

    # List of completed sections
    completed_sections = state["completed_sections"]

    # Format completed section to str to use as context for final sections
    completed_report_sections = "\n\n---\n\n".join(completed_sections)

    return {"final_report": completed_report_sections}


# Conditional edge function to create llm_call workers that each write a section of the report
def assign_workers(state: State):
    """Assign a worker to each section in the plan"""

    # Kick off section writing in parallel via Send() API
    return [Send("llm_call", {"section": s}) for s in state["sections"]]


# Build workflow
orchestrator_worker_builder = StateGraph(State)

# Add the nodes
orchestrator_worker_builder.add_node("orchestrator", orchestrator)
orchestrator_worker_builder.add_node("llm_call", llm_call)
orchestrator_worker_builder.add_node("synthesizer", synthesizer)

# Add edges to connect nodes
orchestrator_worker_builder.add_edge(START, "orchestrator")
orchestrator_worker_builder.add_conditional_edges(
    "orchestrator", assign_workers, ["llm_call"]
)
orchestrator_worker_builder.add_edge("llm_call", "synthesizer")
orchestrator_worker_builder.add_edge("synthesizer", END)

# Compile the workflow
orchestrator_worker = orchestrator_worker_builder.compile()


# Invoke
state = orchestrator_worker.invoke({"topic": "Create a report on LLM scaling laws"})

from IPython.display import Markdown
Markdown(state["final_report"])

# Overview of LLMs and Scaling Laws  

Large Language Models (LLMs) are deep learning models trained on vast amounts of text data to perform a wide range of natural language processing (NLP) tasks, including text generation, translation, summarization, and question answering. These models, such as OpenAI's GPT series, Google's PaLM, and Meta's LLaMA, leverage transformer architectures to capture complex linguistic patterns and contextual relationships.  

A key factor in the advancement of LLMs is the study of **scaling laws**, which describe how model performance improves with increases in model size, dataset size, and computational resources. Empirical research has shown that LLMs exhibit predictable improvements in capabilities when scaled along these dimensions, following power-law relationships. Understanding these scaling laws is critical for optimizing resource allocation, improving efficiency, and guiding future model development.  

Scaling laws also highlight trade-offs between model size, training cost, and performance, influencing decisions in both academic research and industry applications. By leveraging these principles, researchers can design more capable and cost-effective models, pushing the boundaries of what LLMs can achieve.

---

# Understanding Scaling Laws  

Scaling laws describe the predictable relationships between the performance of large language models (LLMs) and key variables such as model size, dataset size, and computational resources. These laws provide a mathematical framework for understanding how improvements in these factors influence model capabilities.  

### Mathematical Foundations  
Scaling laws are often expressed as power-law relationships of the form:  

\[
L(N, D) \approx \left( \frac{N_c}{N} \right)^{\alpha_N} + \left( \frac{D_c}{D} \right)^{\alpha_D}
\]  

Where:  
- \( L \) is the loss (a measure of model error).  
- \( N \) is the number of model parameters.  
- \( D \) is the dataset size.  
- \( N_c, D_c \) are critical thresholds beyond which scaling becomes less effective.  
- \( \alpha_N, \alpha_D \) are scaling exponents that determine the rate of improvement.  

Empirical studies have shown that increasing \( N \) and \( D \) in tandem leads to consistent reductions in loss, with diminishing returns observed once either parameter exceeds its critical threshold.  

### Empirical Observations  
Key findings from scaling law research include:  
1. **Compute-Optimal Scaling**: Performance improves predictably with increased compute, following a power-law trend.  
2. **Data Efficiency**: Larger models require proportionally more data to avoid overfitting.  
3. **Emergent Abilities**: Some capabilities appear only beyond certain scale thresholds, suggesting discontinuous improvements.  
4. **Trade-offs**: Optimal scaling balances model size, data quantity, and training compute, often favoring larger models when resources permit.  

These principles guide the development of more efficient and capable LLMs, enabling better resource allocation and performance prediction.

---

# Key Findings in LLM Scaling  

### **1. Power-Law Scaling Between Model Size and Performance**  
Research has consistently shown that large language model (LLM) performance follows a power-law relationship with model size (parameters), compute budget, and dataset size. Key studies (e.g., Kaplan et al., 2020) demonstrate that:  
- **Performance scales predictably** with increasing parameters, following a smooth power-law curve.  
- **Diminishing returns occur** beyond a certain scale, where further increases in model size yield smaller improvements.  

### **2. Data Scaling is Equally Critical**  
- **Optimal dataset size** scales with model size—larger models require proportionally more training data to avoid underfitting.  
- **Data quality and diversity** significantly impact performance, with some studies suggesting that filtering low-quality data improves efficiency.  

### **3. Compute-Optimal Training**  
- The **Chinchilla scaling laws** (Hoffmann et al., 2022) challenge earlier assumptions, showing that smaller models trained on more data can outperform larger models with less data, given the same compute budget.  
- **Optimal training regimes** balance model size, dataset size, and compute, rather than simply maximizing parameters.  

### **4. Emergent Abilities at Scale**  
- Certain capabilities (e.g., reasoning, few-shot learning) **emerge only beyond a critical scale**, suggesting discontinuous improvements in performance.  
- Larger models exhibit **better sample efficiency**, requiring fewer examples for in-context learning.  

### **5. Efficiency Improvements Through Scaling**  
- **Sparse models and mixture-of-experts (MoE)** architectures demonstrate that not all parameters need to be active during inference, improving efficiency without sacrificing performance.  
- **Scaling laws generalize across architectures**, though variations exist based on model design (e.g., transformers vs. recurrent networks).  

### **6. Challenges in Extreme Scaling**  
- **Training instability** increases with model size, requiring techniques like better initialization and adaptive optimization.  
- **Energy and cost constraints** make extreme scaling economically and environmentally challenging.  

These findings highlight the importance of balancing model size, data, and compute to optimize performance while managing practical constraints.

---

# Implications of Scaling Laws  

Scaling laws have profound implications for the design, training, and deployment of large language models (LLMs), influencing cost, efficiency, and performance trade-offs.  

### **Design Considerations**  
Scaling laws suggest that model performance improves predictably with increases in compute, dataset size, and model parameters. This encourages the development of larger architectures but also necessitates careful resource allocation. Designers must balance:  
- **Parameter Efficiency**: Larger models require more memory and compute, driving innovations in sparse architectures or mixture-of-experts (MoE) models.  
- **Hardware Constraints**: Scaling demands specialized infrastructure (e.g., GPUs/TPUs), influencing architectural choices like parallelism strategies (data, model, or pipeline parallelism).  

### **Training Costs and Efficiency**  
Training LLMs at scale incurs exponential costs in compute and energy. Key trade-offs include:  
- **Diminishing Returns**: Performance gains slow as models grow, requiring optimization techniques like curriculum learning or better data filtering.  
- **Budget Constraints**: Organizations must weigh the benefits of larger models against prohibitive training expenses, leading to increased interest in efficient pretraining (e.g., Chinchilla’s compute-optimal scaling).  

### **Deployment Challenges**  
Scaling laws also affect real-world usability:  
- **Inference Costs**: Larger models demand more resources for inference, prompting techniques like quantization, distillation, or on-demand scaling.  
- **Latency vs. Accuracy**: High-parameter models may offer better accuracy but suffer from slower response times, necessitating trade-offs for latency-sensitive applications.  

### **Future Directions**  
As scaling continues, research focuses on:  
- **Sustainable Scaling**: Reducing energy consumption via efficient architectures.  
- **Alternative Paradigms**: Exploring smaller, specialized models or retrieval-augmented approaches to mitigate scaling costs.  

Ultimately, scaling laws guide—but also constrain—LLM development, requiring a strategic balance between performance gains and practical feasibility.

---

# Challenges and Limitations  

Scaling laws, while powerful for improving model performance, come with several challenges and limitations that must be carefully considered.  

### Diminishing Returns  
As models grow larger, the marginal gains in performance often decrease. Early scaling may yield significant improvements, but beyond a certain point, increasing parameters or data leads to only incremental benefits. This phenomenon raises questions about the cost-effectiveness of further scaling, especially given the substantial resources required.  

### Computational Constraints  
Training large-scale models demands immense computational power, energy, and infrastructure. The costs associated with hardware, electricity, and cooling can be prohibitive for many organizations. Additionally, longer training times and the need for specialized hardware (e.g., GPUs, TPUs) create barriers to entry for smaller research groups and institutions.  

### Ethical Considerations  
The environmental impact of training large models is a growing concern, as energy consumption contributes to carbon emissions. Furthermore, scaling can amplify biases present in training data, leading to unfair or harmful outputs. The concentration of AI development in a few well-resourced entities also raises concerns about accessibility and equitable distribution of benefits.  

### Practical Deployment Challenges  
Even if a model performs well in controlled settings, deploying it in real-world applications may introduce latency, memory constraints, and maintenance difficulties. Smaller, more efficient models are often preferred for production environments, highlighting a trade-off between scale and usability.  

Addressing these challenges requires balancing performance gains with sustainability, fairness, and practical feasibility. Future research must explore techniques like efficient architectures, data pruning, and ethical AI practices to mitigate these limitations.

---

# Future Directions  

Potential future research directions and innovations in LLM scaling encompass several emerging trends and alternative approaches aimed at improving efficiency, performance, and accessibility.  

### **Alternative Scaling Approaches**  
1. **Sparse Models & Mixture-of-Experts (MoE):**  
   - Investigating dynamic routing mechanisms to activate only relevant model components, reducing computational costs.  
   - Optimizing expert selection and load balancing for improved inference efficiency.  

2. **Efficient Training Paradigms:**  
   - Exploring curriculum learning and progressive training strategies to enhance model convergence.  
   - Investigating data pruning and importance sampling to reduce redundant training data.  

3. **Hardware-Aware Optimization:**  
   - Developing specialized architectures (e.g., neuromorphic computing) for LLM inference.  
   - Leveraging quantization, distillation, and sparsity to enable deployment on edge devices.  

### **Emerging Trends**  
1. **Multimodal Scaling:**  
   - Extending LLMs to integrate vision, audio, and other modalities for richer contextual understanding.  
   - Investigating cross-modal attention mechanisms for seamless multimodal reasoning.  

2. **Decentralized & Federated Learning:**  
   - Enabling collaborative training across distributed datasets while preserving privacy.  
   - Exploring blockchain-based incentives for decentralized model contributions.  

3. **Self-Improving Models:**  
   - Developing LLMs capable of iterative self-reflection and refinement via reinforcement learning.  
   - Investigating automated hyperparameter tuning and architecture search.  

4. **Ethical & Sustainable Scaling:**  
   - Reducing carbon footprints via energy-efficient training methods.  
   - Implementing fairness-aware scaling to mitigate biases in large models.  

These directions aim to push the boundaries of LLM capabilities while addressing computational, ethical, and practical challenges.

---

# Conclusion  

Scaling laws have emerged as a foundational principle in the development of artificial intelligence, particularly for large language models (LLMs). The empirical relationship between model size, dataset size, compute resources, and performance has provided a roadmap for optimizing AI systems. Key takeaways include:  

1. **Predictable Performance Gains** – Scaling laws demonstrate that increasing model parameters, training data, or compute yields consistent improvements in performance, enabling more accurate forecasting of resource requirements.  
2. **Diminishing Returns** – While scaling remains effective, the marginal gains decrease as models grow, necessitating more efficient architectures and training techniques.  
3. **Broader Implications for AI Research** – These laws influence not only LLMs but also multimodal and domain-specific models, reinforcing the importance of systematic scaling in AI advancement.  
4. **Economic and Ethical Considerations** – The resource-intensive nature of scaling raises concerns about accessibility, environmental impact, and the concentration of AI development in well-funded organizations.  

Moving forward, the field must balance continued scaling with innovations in efficiency, interpretability, and equitable access to ensure sustainable progress in AI. Scaling laws will remain a critical tool, but their limitations highlight the need for complementary breakthroughs in model design and training methodologies.

In [10]:
# Graph state
class State(TypedDict):
    joke: str
    topic: str
    feedback: str
    funny_or_not: str


# Schema for structured output to use in evaluation
class Feedback(BaseModel):
    grade: Literal["funny", "not funny"] = Field(
        description="Decide if the joke is funny or not.",
    )
    feedback: str = Field(
        description="If the joke is not funny, provide feedback on how to improve it.",
    )


# Augment the LLM with schema for structured output
evaluator = llm.with_structured_output(Feedback)


# Nodes
def llm_call_generator(state: State):
    """LLM generates a joke"""

    if state.get("feedback"):
        msg = llm.invoke(
            f"Write a joke about {state['topic']} but take into account the feedback: {state['feedback']}"
        )
    else:
        msg = llm.invoke(f"Write a joke about {state['topic']}")
    return {"joke": msg.content}


def llm_call_evaluator(state: State):
    """LLM evaluates the joke"""

    grade = evaluator.invoke(f"Grade the joke {state['joke']}")
    return {"funny_or_not": grade.grade, "feedback": grade.feedback}


# Conditional edge function to route back to joke generator or end based upon feedback from the evaluator
def route_joke(state: State):
    """Route back to joke generator or end based upon feedback from the evaluator"""

    if state["funny_or_not"] == "funny":
        return "Accepted"
    elif state["funny_or_not"] == "not funny":
        return "Rejected + Feedback"


# Build workflow
optimizer_builder = StateGraph(State)

# Add the nodes
optimizer_builder.add_node("llm_call_generator", llm_call_generator)
optimizer_builder.add_node("llm_call_evaluator", llm_call_evaluator)

# Add edges to connect nodes
optimizer_builder.add_edge(START, "llm_call_generator")
optimizer_builder.add_edge("llm_call_generator", "llm_call_evaluator")
optimizer_builder.add_conditional_edges(
    "llm_call_evaluator",
    route_joke,
    {  # Name returned by route_joke : Name of next node to visit
        "Accepted": END,
        "Rejected + Feedback": "llm_call_generator",
    },
)

# Compile the workflow
optimizer_workflow = optimizer_builder.compile()

# Invoke
state = optimizer_workflow.invoke({"topic": "Cats"})
print(state["joke"])

Sure! Here's a purr-fect cat joke for you:  

**Why don’t cats play poker in the wild?**  
*Because there are too many cheetahs!* 🐆😹  

(Or if you prefer a classic: **"Why was the cat sitting on the computer?** *To keep an eye on the mouse!*" 🖱️🐱)  

Let me know if you want more—I've got a *litter* of them! 😸


In [11]:
from langchain_core.tools import tool


# Define tools
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b


# Augment the LLM with tools
tools = [add, multiply, divide]
tools_by_name = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)

In [13]:
from langgraph.graph import MessagesState
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage


# Nodes
def llm_call(state: MessagesState):
    """LLM decides whether to call a tool or not"""

    return {
        "messages": [
            llm_with_tools.invoke(
                [
                    SystemMessage(
                        content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
                    )
                ]
                + state["messages"]
            )
        ]
    }


def tool_node(state: dict):
    """Performs the tool call"""

    result = []
    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    return {"messages": result}


# Conditional edge function to route to the tool node or end based upon whether the LLM made a tool call
def should_continue(state: MessagesState) -> Literal["environment", END]:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""

    messages = state["messages"]
    last_message = messages[-1]
    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "Action"
    # Otherwise, we stop (reply to the user)
    return END


# Build workflow
agent_builder = StateGraph(MessagesState)

# Add nodes
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("environment", tool_node)

# Add edges to connect nodes
agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    {
        # Name returned by should_continue : Name of next node to visit
        "Action": "environment",
        END: END,
    },
)
agent_builder.add_edge("environment", "llm_call")

# Compile the agent
agent = agent_builder.compile()


# Invoke
messages = [HumanMessage(content="Add 3 and 4.")]
messages = agent.invoke({"messages": messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================
Tool Calls:
  add (call_0_ca232637-dae6-4778-866d-9f436baacc30)
 Call ID: call_0_ca232637-dae6-4778-866d-9f436baacc30
  Args:
    a: 3
    b: 4
================================= Tool Message =================================

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.


In [14]:
from langgraph.prebuilt import create_react_agent

# Pass in:
# (1) the augmented LLM with tools
# (2) the tools list (which is used to create the tool node)
pre_built_agent = create_react_agent(llm, tools=tools)

# Invoke
messages = [HumanMessage(content="Add 3 and 4.")]
messages = pre_built_agent.invoke({"messages": messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================
Tool Calls:
  add (call_0_9a3550fe-d693-4400-afdd-7589f5014d96)
 Call ID: call_0_9a3550fe-d693-4400-afdd-7589f5014d96
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.


In [15]:
import os
import shutil
import sqlite3

import pandas as pd
import requests

db_url = "https://storage.googleapis.com/benchmarks-artifacts/travel-db/travel2.sqlite"
local_file = "travel2.sqlite"
# The backup lets us restart for each tutorial section
backup_file = "travel2.backup.sqlite"
overwrite = False
if overwrite or not os.path.exists(local_file):
    response = requests.get(db_url)
    response.raise_for_status()  # Ensure the request was successful
    with open(local_file, "wb") as f:
        f.write(response.content)
    # Backup - we will use this to "reset" our DB in each section
    shutil.copy(local_file, backup_file)


# Convert the flights to present time for our tutorial
def update_dates(file):
    shutil.copy(backup_file, file)
    conn = sqlite3.connect(file)
    cursor = conn.cursor()

    tables = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table';", conn
    ).name.tolist()
    tdf = {}
    for t in tables:
        tdf[t] = pd.read_sql(f"SELECT * from {t}", conn)

    example_time = pd.to_datetime(
        tdf["flights"]["actual_departure"].replace("\\N", pd.NaT)
    ).max()
    current_time = pd.to_datetime("now").tz_localize(example_time.tz)
    time_diff = current_time - example_time

    tdf["bookings"]["book_date"] = (
        pd.to_datetime(tdf["bookings"]["book_date"].replace("\\N", pd.NaT), utc=True)
        + time_diff
    )

    datetime_columns = [
        "scheduled_departure",
        "scheduled_arrival",
        "actual_departure",
        "actual_arrival",
    ]
    for column in datetime_columns:
        tdf["flights"][column] = (
            pd.to_datetime(tdf["flights"][column].replace("\\N", pd.NaT)) + time_diff
        )

    for table_name, df in tdf.items():
        df.to_sql(table_name, conn, if_exists="replace", index=False)
    del df
    del tdf
    conn.commit()
    conn.close()

    return file


db = update_dates(local_file)